# Math to determine how far outposts can be.

Let:
- W be wagon capacity
- r be target throughput (rate)
- n be number of trains
- r_u be unload rate

These we can control.

In [1]:
from typing import Iterable

from sympy import *

from focus import focus

init_printing()


In [2]:
# setup equations
global_eqs: set[Eq] = set()

In [3]:
import re


def get_eq(s: str) -> Eq:
    if '<' in s or '>' in s:
        # get left, rel, right
        t = re.split(r'((?:<|>)=?)', s)
        print(t)
        left, rel, right = t
        s = Rel(sympify(left), sympify(right), rel)
    elif '=' in s:
        left, right = s.split('=')
        s = Eq(sympify(left), sympify(right))
    else:
        raise ValueError(f'Invalid equation: {s}')
    pprint(s)
    return s


def get_eqs(*strs: str) -> list[Eq]:
    return [get_eq(s) for s in strs]


def add_eq(s: str) -> Eq:
    result = get_eq(s)
    global_eqs.add(result)
    return result



Define some other values:
- Let t_u be time to unload: t_u = W / r_u
- Let C be cycle time (time for train to go to outpost and back).
- Cycle time consists of traveling time (T) and time unloading (t_u). T + t_u equals _half_ the cycle time (one way). We also add some constant factor for trains needing to slow down and acceleration (t_k).

Our goal is to find 

In [4]:
add_eq("t_u=W/r_u")
add_eq("C=2*(T+t_u)")


     W 
tᵤ = ──
     rᵤ
C = 2⋅T + 2⋅tᵤ


In [5]:
free = ['t_u', 'C']


# helper to solve for a variable
def solvev(syms, *extra_eqs, **kwargs):
    if not isinstance(syms, Iterable):
        sym_set = {Symbol(syms)}
    else:
        sym_set = {Symbol(s) if isinstance(s, str) else s for s in syms}
    all_eqs = global_eqs | set(extra_eqs)
    all_syms = Tuple(*all_eqs).free_symbols
    all_free = (set(symbols(free)) | sym_set) & all_syms
    res = focus(all_eqs, *all_free, **kwargs)
    if isinstance(res, dict):
        res = {
            sym.name: simplify(Eq(sym, sol))
            for sym, sol in res.items()
            if sym in sym_set
        }
        if len(res) == 1:
            return next(iter(res.values()))
    return res


## Max outpost distance

To meet our target throughput, we need items delivery rate to match: r = (W/C) * n; throughput per train * num trains


In [6]:
target_throughput = get_eq("r=n*W/C")
pprint(solvev('T', target_throughput))
# result:
# T = W( n/2r - 1/r_u) )


    W⋅n
r = ───
     C 
    W⋅n   W 
T = ─── - ──
    2⋅r   rᵤ


What this shows:
- Increasing wagon capacity (more wagons, or on site smelting) almost directly increases Travel time (minus a constant factor).
- Increasing unload rate has quickly diminishing returns (1/r_u).
- More trains is very slightly better than more wagons directly, until you are not able to unload fast enough.

# Only 1 train unloading: 
To ensure that there's only ever 1 train unloading at a time.
Let v be average number of trains loading at any given time.
Trains spend (2t_u/C) of their time loading. There are 2 loading stations, n trains, so n*(2t_u/C) <= 2:

In [7]:
eq = get_eq("n*t_u/C=v")
res = solvev('n', eq, target_throughput)
pprint(res)

pprint(solvev('r', eq, target_throughput))
# 
# min_r = solve(res, 'r_u')[0]
# pprint(min_r)
# 
# min_r_ineq = Eq(Symbol('r_u'), min_r.xreplace({Symbol('T'): max_t.rhs}))
# 
# print(min_r_ineq)
# res = solve(min_r_ineq, 'r_u')
# pprint(res)

# result:
# n = r/v


n⋅tᵤ    
──── = v
 C      
n = 0
r = rᵤ⋅v


So, as it turns out, all we need is that the unload rate be larger than the target throughput.

# Max distance for 3 trains, 1 shared track

With 1 shared track, there can only be one train going along the track any time.
So, at best, we have 2 trains loading at each station, and 1 traveling.
This makes it about equivalent to only 1 train, with instant unloading.

Let's look at T in this case:

In [8]:
eqs = get_eqs("r_u = oo", "n=1")

res_p2p = solvev(['T'], *eqs, target_throughput)
pprint(res_p2p)

# result: T = W/2r - k


rᵤ = ∞
n = 1
     W 
T = ───
    2⋅r


So, the max distance is only a function of train capacity.

Comparing to if we had 2 tracks, one for each direction:


In [9]:
eqs = get_eqs("n=3")
res_3_train: Eq = solvev(['T'], *eqs, target_throughput)
pprint(res_3_train)

diff: Expr = res_3_train.rhs - res_p2p.rhs
pprint(diff)
# result: W (1/r - 1/r_u)

n = 3
      W    3⋅W
T = - ── + ───
      rᵤ   2⋅r
  W    W
- ── + ─
  rᵤ   r


In [10]:
# evaluate that with some realistic numbers
sub = {
    'W': 40 * 50,
    'r': 30,
    'r_u': 8 * 4.62,
    'n': 3,
}

# Some evaluations

In [18]:
# p2p
print("8 unload inserters per wagon, target 1 red belt per wagon")

print("p2p (shared rail):")
pprint(res_p2p.rhs.evalf(subs=sub))

print("3 trains, 2 way:")
pprint(res_3_train.rhs.evalf(subs=sub))

print("3 trains, 2 way, 10 unload inserters:")
pprint(res_3_train.rhs.evalf(subs={
    **sub,
    'r_u': 10 * 4.62
}))

print("2 trains, on-site smelting:")
pprint(solvev('T', target_throughput).rhs.evalf(subs={
    **sub,
    'W': 40 * 100,
    'n': 2,
}))

print("3 trains, on-site:")
pprint(solvev('T', target_throughput).rhs.evalf(subs={
    **sub,
    'W': 40 * 100,
    'n': 3,
}))

print("2 trains, on-site, 10 unload inserters:")
pprint(solvev('T', target_throughput).rhs.evalf(subs={
    **sub,
    'W': 40 * 100,
    'n': 2,
    'r_u': 10 * 4.62,
}))

print("3 trains, on-site, 10 unload inserters:")
pprint(solvev('T', target_throughput).rhs.evalf(subs={
    **sub,
    'W': 40 * 100,
    'n': 3,
    'r_u': 10 * 4.62,
}))



8 unload inserters per wagon, target 1 red belt per wagon
p2p (shared rail):
33.3333333333333
3 trains, 2 way:
45.8874458874459
3 trains, 2 way, 10 unload inserters:
56.7099567099567
2 trains, on-site smelting:
25.1082251082251
3 trains, on-site:
91.7748917748918
2 trains, on-site, 10 unload inserters:
46.7532467532468
3 trains, on-site, 10 unload inserters:
113.419913419913


On-site smelting only makes sense if you have at least 10 unload inserters (unload speed >> throughput).
Otherwise, with 3 trains, its much more costly and not much benefit, and with 2 trains the distance sucks.

TODO: more detailed comparison (full builds, not just outpost distance) with:
- On site smelting vs not
- Train size (# of wagons). In theory, more is better?
- Train amount
- number of unload inserters.